In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
import pickle

In [ ]:
df = pd.read_csv('../Resources/card_transdata.csv')
df.head()

In [ ]:
df.columns

In [ ]:
conti_variable = df[['distance_from_home', 'distance_from_last_transaction', 'ratio_to_median_purchase_price']]
conti_variable.head()

In [ ]:
conti_variable.hist(bins=60)

In [ ]:
def removeOutliers(data, col):
    Q3 = np.quantile(data[col], 0.75)
    Q1 = np.quantile(data[col], 0.25)
    IQR = Q3 - Q1
 
    print("IQR value for column %s is: %s" % (col, IQR))
    global outlier_free_list
    global filtered_data
 
    lower_range = Q1 - 1.5 * IQR
    upper_range = Q3 + 1.5 * IQR
    outlier_free_list = [x for x in data[col] if (
        (x > lower_range) & (x < upper_range))]
    filtered_data = data.loc[data[col].isin(outlier_free_list)]

In [ ]:
for x in conti_variable.columns:
    q25, q75 = conti_variable[x].quantile(0.25), conti_variable[x].quantile(0.75)
    iqr = q75 - q25
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off

    conti_variable[x+'_outlier'] = np.where((conti_variable[x]<lower) & (conti_variable[x]<upper),1,0) 
    
conti_variable.head()    

# Unbalanced Data

In [ ]:
sns.countplot(x='fraud',data=df)

In [ ]:
df = df.sample(frac=1)

fraud_df = df.loc[df['fraud'] == 1]
non_fraud_df = df.loc[df['fraud'] == 0][:87403]

normal_distributed_df = pd.concat([fraud_df, non_fraud_df]).sample(frac=1, random_state=42)

# Shuffle dataframe rows
new_df = normal_distributed_df.sample(frac=1, random_state=42)

new_df.head()

In [ ]:
print('Distribution of the Classes in the subsample dataset')
print(new_df['fraud'].value_counts()/len(new_df))

sns.countplot('fraud', data=new_df)
plt.title('Equally Distributed Classes')
plt.show()

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(24,20))

# Entire DataFrame
corr = df.corr()
sns.heatmap(corr, cmap='coolwarm_r', annot_kws={'size':20}, ax=ax1)
ax1.set_title("Imbalanced Correlation Matrix \n (don't use for reference)", fontsize=14)


sub_sample_corr = new_df.corr()
sns.heatmap(sub_sample_corr, cmap='coolwarm_r', annot_kws={'size':20}, ax=ax2)
ax2.set_title('SubSample Correlation Matrix \n (use for reference)', fontsize=14)
plt.show()

# Logistic

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
x = new_df.drop(['fraud'],axis=1)

In [ ]:
y = new_df['fraud']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [ ]:
logistic = LogisticRegression(max_iter=1000)

In [ ]:
model = logistic.fit(x_train, y_train)

In [ ]:
prediction = model.predict(x_test)

In [ ]:
print("Accuracy - " + str(accuracy_score(y_test,prediction)))
print("Recall - " + str(recall_score(y_test,prediction)))
print("precision - " + str(precision_score(y_test,prediction)))

In [ ]:
pickle.dump(model, open('../Models/US_LogisticReg_08132022.pkl', 'wb'))

In [ ]:
plot_confusion_matrix(logistic,x_test,y_test)

# Random Classifier

In [ ]:
x1 = new_df.drop(['fraud'],axis=1)

In [ ]:
y1 = new_df['fraud']

In [ ]:
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, random_state=42)

In [ ]:
randomforest = RandomForestClassifier()

In [ ]:
model1 = randomforest.fit(x1_train,y1_train)

In [ ]:
prediction1 = model1.predict(x1_test)

In [ ]:
accuracy_score(y1_test,prediction1)

In [ ]:
print("Accuracy - " + str(accuracy_score(y1_test,prediction1)))
print("Recall - " + str(recall_score(y1_test,prediction1)))
print("precision - " + str(precision_score(y1_test,prediction1)))

In [ ]:
pickle.dump(model1, open('../Models/US_RandomForest_08132022.pkl', 'wb'))

In [ ]:
plot_confusion_matrix(randomforest,x1_test,y1_test)

# Decision Tree

In [ ]:
x2 = new_df.drop(['fraud'],axis=1)

In [ ]:
y2 = new_df['fraud']

In [ ]:
dt = DecisionTreeRegressor()

In [ ]:
x2_train,x2_test,y2_train,y2_test=train_test_split(x2,y2,test_size=0.3,random_state=123)

In [ ]:
model2 = dt.fit(x2_train,y2_train)

In [ ]:
prediction2 = model2.predict(x2_test)

In [ ]:
print("Accuracy - " + str(accuracy_score(y2_test,prediction2)))
print("Recall - " + str(recall_score(y2_test,prediction2)))
print("precision - " + str(precision_score(y2_test,prediction2)))

In [ ]:
pickle.dump(model2, open('../Models/US_DecisionTree_08132022.pkl', 'wb'))